In [1]:
from neuron import h
import ipywidgets as widgets
from ipywidgets import HBox,VBox,Label,Layout
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
h.load_file('stdrun.hoc')

1.0

In [2]:
soma = h.Section(name='soma')
soma.L = 100 # um
soma.insert('hh')
v0_vec = h.Vector()
t_vec = h.Vector()
v0_vec.record(soma(0.5)._ref_v)
t_vec.record(h._ref_t)

1.0

In [3]:
soma.insert('na1')
soma.insert('k1')
m_na = h.Vector()
h_na = h.Vector()
n_k = h.Vector()
tau_m = h.Vector()
tau_h = h.Vector()
tau_n = h.Vector()
m_na.record(soma(0.5)._ref_minf_na1)
h_na.record(soma(0.5)._ref_hinf_na1)
n_k.record(soma(0.5)._ref_ninf_k1)
tau_m.record(soma(0.5)._ref_mtau_na1)
tau_h.record(soma(0.5)._ref_htau_na1)
tau_n.record(soma(0.5)._ref_taun_k1)

1.0

In [4]:
def activemodel(diam,cm,el,gl,gna,gk,tstop,dur,amp,mhalf,hhalf,mk,hk,nhalf,kn,flag_1,flag_2,multiply,multiply1,multiply2):
    soma.diam = diam
    soma.cm = cm*1.4884e-4/6.2832e-4
    soma.gnabar_hh = gna*1e-3
    soma.gkbar_hh = gk*1e-3
    soma.gl_hh = gl*1e-6
    soma.el_hh = el
    soma.mhalf_na1 = mhalf
    soma.hhalf_na1 = hhalf
    soma.mk_na1 = mk
    soma.hk_na1 = hk
    soma.vhalfn_k1 = nhalf
    soma.kn_k1 = kn
    soma.flag_na1 = flag_1
    soma.flag_k1 = flag_2
    soma.multiply1_na1 = multiply1
    soma.multiply2_na1 = multiply2
    soma.multiply_k1 = multiply
        
    
    Source = h.IClamp(soma(0.5))
    Source.delay = dur[0]
    Source.dur = dur[1]-dur[0]
    Source.amp = amp
    
    
    h.tstop = tstop
    h.v_init = el
    
    nc = h.NetCon(soma(0.5)._ref_v,None,sec=soma)
    nc.threshold = 0
    spvec = h.Vector()
    nc.record(spvec)
    
    h.run()
    print('number of spikes:',len(spvec))
    
    
    

    if soma.flag_k1 == 0:
        R_in_1 = 1/(soma.gl_hh*1000)
        tau_1 = soma.cm*R_in_1
        V_rest_1 = v0_vec[int(tstop)-20]
        soma.flag_k1 = 1 - soma.flag_k1
        h.run()
        dt_V = v0_vec[int(dur[1])-10]-v0_vec[int(dur[0])-10]
        R_in_2 = dt_V/Source.amp
        tau_2 = soma.cm*R_in_2
        V_rest_2 = v0_vec[int(tstop)-20]
        soma.flag_k1 = 1 - soma.flag_k1
        h.run()

    if soma.flag_k1 == 1:
        dt_V = v0_vec[int(dur[1])-10]-v0_vec[int(dur[0])-10]
        R_in_2 = dt_V/Source.amp
        tau_2 = soma.cm*R_in_2
        V_rest_2 = v0_vec[int(tstop)-20]
        soma.flag_k1 = 1 - soma.flag_k1
        h.run()
        R_in_1 = 1/(soma.gl_hh*1000)
        tau_1 = soma.cm*R_in_1
        V_rest_1 = v0_vec[int(tstop)-20]
        soma.flag_k1 = 1 - soma.flag_k1
        h.run()
            
            
    df1 = pd.DataFrame([[tau_1],[V_rest_1],[R_in_1],[tau_2],[V_rest_2],[R_in_2]],
    columns=['value  '],
    index = ['tau_seg  ', 'Vrest_seg  ', 'Rin_seg  ','tau_ori  ', 'Vrest_ori  ', 'Rin_ori  '])
    if Source.amp < 0:
        print (df1)
        
    plt.figure(figsize=(16,14))
    plt.subplot(4,1,1)
    plt.plot(t_vec, v0_vec,'b')
    plt.xlim(0, tstop)
    plt.ylabel('mV')
        
    # FIR Curve

    freq1 = []
    I1 = []
    freq2 = []
    I2 = []
    if (Source.amp > 0):
        for Source.amp in np.arange(amp,amp+3,0.2):
            h.run()
            freq1.append(len(spvec)/Source.dur)
            I1.append(Source.amp)
        soma.flag_k1 = 1 - soma.flag_k1
        for Source.amp in np.arange(amp,amp+3,0.2):    
            h.run()
            freq2.append(len(spvec)/Source.dur)
            I2.append(Source.amp)
        plt.subplot(4,1,2)
        plt.plot(I1,freq1,'yo')
        plt.plot(I2,freq2,'bo')
        plt.legend([soma.flag_k1,1-soma.flag_k1])
        plt.xlabel('Injected current (nA)')
        plt.ylabel('frequency') 
        plt.subplot(4,1,3)
        plt.plot(v0_vec, m_na,'r.')
        plt.plot(v0_vec, h_na,'b.')
        plt.plot(v0_vec, n_k, 'g.')
       # plt.xlim(0, tstop)
        plt.xlabel('voltage (mV)')
        plt.ylabel('Probability')
        plt.legend(['minf','hinf','ninf'])
        plt.subplot(4,1,4)
        plt.plot(v0_vec, tau_m,'r.')
        plt.plot(v0_vec, tau_h,'b.')
        plt.plot(v0_vec, tau_n, 'g.')
        plt.xlabel('vlotage (mV)')
        plt.ylabel('tau')
        plt.legend(['minf','hinf','ninf'])
    plt.show()

In [8]:
# default settings
diam = 200
cm = 1
el = -70
gl = 30
gna = 120
gk = 12
tstop = 500
dur = [100,400]
amp = 0.1
mhalf = -38
hhalf = -50
mk = 7
hk = 4
kn = 11
vhalfn = -9
flag_1 = 0
flag_2 = 0
multiply = 1
multiply1 = 1
multiply2 = 1



w_model = widgets.Checkbox(value=False,description='Easy_Model',button_style='info')
w_pass = widgets.Checkbox(value=False,description='Passive',button_style='info')
w_el = widgets.FloatSlider(value=el,min=-80,max=-60,step=.2,continuous_update=False,readout_format='.1f')
w_gl = widgets.FloatSlider(value=gl,min=5,max=50,step=.2,continuous_update=False,readout_format='.1f')
w_gna = widgets.FloatSlider(value=gna,min=0,max=200,step=.5,continuous_update=False,readout_format='.1f')
w_gk = widgets.FloatSlider(value=gk,min=0,max=30,step=.1,continuous_update=False,readout_format='.1f')
w_mhalf = widgets.FloatSlider(value=mhalf,min=-50,max=-5,step=1,continuous_update=False,readout_format='.1f')
w_hhalf = widgets.FloatSlider(value=hhalf,min=-80,max=-40,step=1,continuous_update=False,readout_format='.1f')
w_nhalf = widgets.FloatSlider(value=vhalfn,min=-30,max=5,step=1,continuous_update=False,readout_format='.1f')
w_flag_1 = widgets.FloatSlider(value=flag_1,min=0,max=1,step=1,continuous_update=False,readout_format='.1f')

w_tstop = widgets.FloatText(value=500)
w_dur = widgets.FloatRangeSlider(value=[100,400],min=0,max=500,step=5,continuous_update=False,readout_format='.0f')
w_amp = widgets.FloatSlider(value=0.1,min=-.2,max=.8,step=.01,continuous_update=False,readout_format='.2f')
w_diam = widgets.FloatSlider(value=diam,min=100,max=300,step=2,continuous_update=False,readout_format='.0f')
w_cm = widgets.FloatLogSlider(value=cm,min=-1,max=1,step=.05,continuous_update=False,readout_format='.1f')
w_mk = widgets.FloatSlider(value=mk,min=0,max=30,step=.1,continuous_update=False,readout_format='.1f')
w_hk = widgets.FloatSlider(value=hk,min=0,max=30,step=.1,continuous_update=False,readout_format='.1f')
w_kn = widgets.FloatSlider(value=kn,min=0,max=30,step=1,continuous_update=False,readout_format='.1f')
w_multiply = widgets.FloatSlider(value=multiply,min=0,max=1,step=.1,continuous_update=False,readout_format='.1f')
w_multiply1 = widgets.FloatSlider(value=multiply1,min=0,max=1,step=.1,continuous_update=False,readout_format='.1f')
w_multiply2 = widgets.FloatSlider(value=multiply2,min=0,max=1,step=.1,continuous_update=False,readout_format='.1f')
w_flag_2 = w_flag_1 

def update_pass(*args):
    if w_pass.value:
        global gna,gk,df1
        gna = w_gna.value
        gk = w_gk.value
        w_amp.value = -0.03
        w_gna.value = w_gk.value = 0
        w_gna.disabled = w_gk.disabled = True
        
    else:
        w_gna.value = gna
        w_gk.value = gk
        w_amp.value = 0.1
        w_gna.disabled = w_gk.disabled = False
w_pass.observe(update_pass,'value')

def update_easy(*args):
    if w_model.value:
        global mhalf,hhalf,vhalfn,mk,hk,kn,multiply,multiply1,multiply2
        mhalf = w_mhalf 
        hhalf = w_hhalf
        vhalf = w_nhalf
        mk = w_mk 
        hk = w_hk
        kn = w_kn 
        multiply = w_multiply
        multiply1 = w_multiply1
        multiply2 = w_multiply2
        w_mhalf.disabled = True
        w_hhalf.disabled = True
        w_nhalf.disabled = True
        w_mk.disabled = True
        w_hk.disabled = True
        w_kn.disabled = True
        w_multiply.disabled = True
        w_multiply1.disabled = True
        w_multiply2.disabled = True
w_model.observe(update_easy,'value')
                 
def update_dur(*args):
    w_dur.max = w_tstop.value
w_tstop.observe(update_dur,'value')


ui = HBox([VBox([Label('Easy Model'),Label('Passive Cell'),Label(value=r'\( E_{l}\:(mV) \)'),Label(value=r'\( g_{leak}\:(\mu S/cm^2) \)'),
                 Label(value=r'\( g_{Na}\:(mS/cm^2) \)'),Label(value=r'\( g_{K}\:(mS/cm^2) \)'),Label(value=r'\( V{1/2m}\: \)'),Label(value=r'\( V{1/2h}\: \)'),Label(value=r'\( V{1/2n}\: \)'),Label(value=r'\(segregation\: \)')],layout=Layout(width='10%')),
           VBox([w_model,w_pass,w_el,w_gl,w_gna,w_gk,w_mhalf,w_hhalf,w_nhalf,w_flag_1],layout=Layout(width='38%')),
           VBox([Label(value=r'\( tstop\:(ms) \)'),Label(value=r'\( Injection duration\:(ms) \)'),Label(value=r'\( I_{inject}\:(nA) \)'),
                 Label(value=r'\( soma\:diameter\:(\mu m) \)'),Label(value=r'\( capacitance\:scale \)'),Label(value=r'\( k_{m}\: \)'),Label(value=r'\( k_{h}\: \)'),Label(value=r'\( k_{n}\: \)'),
                Label(value= 'multiply_n'),Label(value='multiply_m'),Label(value='multiply_h')],layout=Layout(width='14%')),
           VBox([w_tstop,w_dur,w_amp,w_diam,w_cm,w_mk,w_hk,w_kn,w_multiply,w_multiply1,w_multiply2],layout=Layout(width='38%')) ])

out = widgets.interactive_output(activemodel,{'el':w_el,'gl':w_gl,'gna':w_gna,'gk':w_gk,
                                              'tstop':w_tstop,'dur':w_dur,'amp':w_amp,'diam':w_diam,'cm':w_cm,
                                              'mhalf':w_mhalf,'hhalf':w_hhalf,'nhalf':w_nhalf,'mk':w_mk,
                                              'hk':w_hk,'kn':w_kn,'flag_1':w_flag_1,'flag_2':w_flag_2,
                                             'multiply':w_multiply,'multiply1':w_multiply1,'multiply2':w_multiply2})
display(ui,out)


Output()